In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
data = pd.read_csv('laptop_price.csv')
data.head(10)


,Company,Product,TypeName,Inches,ScreenResolution,CPU_Company,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Company,GPU_Type,OpSys,Weight (kg),Price (Euro)
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,2.3,8,128GB SSD,Intel,Iris Plus Graphics 640,macOS,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel,Core i5,1.8,8,128GB Flash Storage,Intel,HD Graphics 6000,macOS,1.34,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel,Core i5 7200U,2.5,8,256GB SSD,Intel,HD Graphics 620,No OS,1.86,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel,Core i7,2.7,16,512GB SSD,AMD,Radeon Pro 455,macOS,1.83,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,3.1,8,256GB SSD,Intel,Iris Plus Graphics 650,macOS,1.37,1803.60
5,Acer,Aspire 3,Notebook,15.6,1366x768,AMD,A9-Series 9420,3.0,4,500GB HDD,AMD,Radeon R5,Windows 10,2.10,400.00
6,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel,Core i7,2.2,16,256GB Flash Storage,Intel,Iris Pro Graphics,Mac OS X,2.04,2139.97
7,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel,Core i5,1.8,8,256GB Flash Storage,Intel,HD Graphics 6000,macOS,1.34,1158.70
8,Asus,ZenBook UX430UN,Ultrabook,14.0,Full HD 1920x1080,Intel,Core i7 8550U,1.8,16,512GB SSD,Nvidia,GeForce MX150,Windows 10,1.30,1495.00
9,Acer,Swift 3,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Intel,Core i5 8250U,1.6,8,256GB SSD,Intel,UHD Graphics 620,Windows 10,1.60,770.00


In [3]:
print(data.shape)
print(data.info())

(1275, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company              1275 non-null   object 
 1   Product              1275 non-null   object 
 2   TypeName             1275 non-null   object 
 3   Inches               1275 non-null   float64
 4   ScreenResolution     1275 non-null   object 
 5   CPU_Company          1275 non-null   object 
 6   CPU_Type             1275 non-null   object 
 7   CPU_Frequency (GHz)  1275 non-null   float64
 8   RAM (GB)             1275 non-null   int64  
 9   Memory               1275 non-null   object 
 10  GPU_Company          1275 non-null   object 
 11  GPU_Type             1275 non-null   object 
 12  OpSys                1275 non-null   object 
 13  Weight (kg)          1275 non-null   float64
 14  Price (Euro)         1275 non-null   float64
dtypes: float64(4), int64(1), ob

# Data preprocessiong

In [4]:
data.isnull().sum()

Company                0
Product                0
TypeName               0
Inches                 0
ScreenResolution       0
CPU_Company            0
CPU_Type               0
CPU_Frequency (GHz)    0
RAM (GB)               0
Memory                 0
GPU_Company            0
GPU_Type               0
OpSys                  0
Weight (kg)            0
Price (Euro)           0
dtype: int64

In [5]:
for col in data.columns:
    print(col,':',data[col].nunique())

Company : 19
Product : 618
TypeName : 6
Inches : 18
ScreenResolution : 40
CPU_Company : 3
CPU_Type : 93
CPU_Frequency (GHz) : 25
RAM (GB) : 9
Memory : 39
GPU_Company : 4
GPU_Type : 106
OpSys : 9
Weight (kg) : 171
Price (Euro) : 791


In [6]:
input = data.drop('Price (Euro)', axis = 1)
output = data['Price (Euro)']

input.head()

,Company,Product,TypeName,Inches,ScreenResolution,CPU_Company,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Company,GPU_Type,OpSys,Weight (kg)
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,2.3,8,128GB SSD,Intel,Iris Plus Graphics 640,macOS,1.37
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel,Core i5,1.8,8,128GB Flash Storage,Intel,HD Graphics 6000,macOS,1.34
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel,Core i5 7200U,2.5,8,256GB SSD,Intel,HD Graphics 620,No OS,1.86
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel,Core i7,2.7,16,512GB SSD,AMD,Radeon Pro 455,macOS,1.83
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,3.1,8,256GB SSD,Intel,Iris Plus Graphics 650,macOS,1.37


In [7]:
numerical_data = input.select_dtypes(exclude = 'object').columns
categorical_data = input.select_dtypes(include = 'object').columns

## Data Encoding

### usnig One-Hot for some nominal data

In [8]:
def one_hot_encode(input_data, column):
    dummies = pd.get_dummies(input_data[column])
    input_data = pd.concat([input_data, dummies], axis=1)
    input_data = input_data.drop(column, axis = 1)
    return input_data


In [9]:
input = one_hot_encode(input, 'Company')
input = one_hot_encode(input, 'TypeName')
input = one_hot_encode(input, 'CPU_Company')
input = one_hot_encode(input, 'GPU_Company')

In [10]:
input.head()

,Product,Inches,ScreenResolution,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Type,OpSys,Weight (kg),...,Notebook,Ultrabook,Workstation,AMD,Intel,Samsung,AMD,ARM,Intel,Nvidia
0,MacBook Pro,13.3,IPS Panel Retina Display 2560x1600,Core i5,2.3,8,128GB SSD,Iris Plus Graphics 640,macOS,1.37,...,False,True,False,False,True,False,False,False,True,False
1,Macbook Air,13.3,1440x900,Core i5,1.8,8,128GB Flash Storage,HD Graphics 6000,macOS,1.34,...,False,True,False,False,True,False,False,False,True,False
2,250 G6,15.6,Full HD 1920x1080,Core i5 7200U,2.5,8,256GB SSD,HD Graphics 620,No OS,1.86,...,True,False,False,False,True,False,False,False,True,False
3,MacBook Pro,15.4,IPS Panel Retina Display 2880x1800,Core i7,2.7,16,512GB SSD,Radeon Pro 455,macOS,1.83,...,False,True,False,False,True,False,True,False,False,False
4,MacBook Pro,13.3,IPS Panel Retina Display 2560x1600,Core i5,3.1,8,256GB SSD,Iris Plus Graphics 650,macOS,1.37,...,False,True,False,False,True,False,False,False,True,False


### using Label Encoding

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in input.select_dtypes('object'):
    input[col] = le.fit_transform(input[col])


In [12]:
input.head()

,Product,Inches,ScreenResolution,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Type,OpSys,Weight (kg),...,Notebook,Ultrabook,Workstation,AMD,Intel,Samsung,AMD,ARM,Intel,Nvidia
0,300,13.3,23,40,2.3,8,4,56,8,1.37,...,False,True,False,False,True,False,False,False,True,False
1,301,13.3,1,40,1.8,8,2,50,8,1.34,...,False,True,False,False,True,False,False,False,True,False
2,50,15.6,8,46,2.5,8,16,52,4,1.86,...,True,False,False,False,True,False,False,False,True,False
3,300,15.4,25,54,2.7,16,29,76,8,1.83,...,False,True,False,False,True,False,True,False,False,False
4,300,13.3,23,40,3.1,8,16,57,8,1.37,...,False,True,False,False,True,False,False,False,True,False


## Data Scaling ( Normalization )

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
input[numerical_data] = scaler.fit_transform(input[numerical_data])

In [14]:
input.head()

,Product,Inches,ScreenResolution,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Type,OpSys,Weight (kg),...,Notebook,Ultrabook,Workstation,AMD,Intel,Samsung,AMD,ARM,Intel,Nvidia
0,300,0.385542,23,40,0.518519,0.096774,4,56,8,0.169576,...,False,True,False,False,True,False,False,False,True,False
1,301,0.385542,1,40,0.333333,0.096774,2,50,8,0.162095,...,False,True,False,False,True,False,False,False,True,False
2,50,0.662651,8,46,0.592593,0.096774,16,52,4,0.291771,...,True,False,False,False,True,False,False,False,True,False
3,300,0.638554,25,54,0.666667,0.225806,29,76,8,0.284289,...,False,True,False,False,True,False,True,False,False,False
4,300,0.385542,23,40,0.814815,0.096774,16,57,8,0.169576,...,False,True,False,False,True,False,False,False,True,False


## Data Spliting
80% for training
20% for Testing

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input, output, test_size = 0.2, random_state = 43)


In [16]:
x_train.head()

,Product,Inches,ScreenResolution,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Type,OpSys,Weight (kg),...,Notebook,Ultrabook,Workstation,AMD,Intel,Samsung,AMD,ARM,Intel,Nvidia
270,300,0.385542,23,40,0.740741,0.096774,29,55,8,0.169576,...,False,True,False,False,True,False,False,False,True,False
860,456,0.662651,15,46,0.592593,0.096774,10,52,5,0.401496,...,True,False,False,False,True,False,False,False,True,False
426,247,0.662651,8,70,0.333333,0.096774,20,74,5,0.331671,...,True,False,False,False,True,False,True,False,False,False
994,243,0.385542,7,46,0.592593,0.096774,16,52,5,0.246883,...,False,False,False,False,True,False,False,False,True,False
1197,392,0.867470,15,58,0.629630,0.225806,31,32,5,0.900249,...,False,False,False,False,True,False,False,False,False,True


In [17]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)
y_test_pred = model.predict(x_test)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 134598.00766063316
R-squared: 0.7706824006555612
